# BUILD CHROMA CQT

In [1]:
import numpy as np
import librosa
import pandas as pd
from sklearn import preprocessing
from tqdm.notebook import trange, tqdm

### GLOBAL VARIABLES

In [2]:
INPUT_PATH = 'data/processed/submission.npy'
OUTPUT_PATH = 'data/features/submission_tonnetz.npy'
SR = 22050

### FUNCTIONS

In [3]:
def wave2tonnetz(wave, sr, normalize, max_pad_len, padding_mode):

    #wave, sr = librosa.load(wav_file, mono=True)
    
    if 0 < len(wave): # workaround: 0 length causes error
        wave, _ = librosa.effects.trim(wave) # trim, top_db=default(60)

    if normalize:
        wave = librosa.util.normalize(wave) # normalizing data before mfcc

    # making melspect from signal
    S = librosa.feature.tonnetz(y=wave, sr=sr)

    if max_pad_len:
        if S.shape[1] > max_pad_len:
            S = S[:,:max_pad_len]
        else:
            pad_width = max_pad_len - S.shape[1]
            S = np.pad(S, pad_width=((0, 0), (0, pad_width)), mode=padding_mode)

   
    return S

In [4]:
def gen_tonnetz(input_path, output_path, sr,  max_pad_len=None, normalize=True, padding_mode='constant'):
    
    # Load wave files
    wavfiles = np.load(input_path)
    
    # Init Mel vectors
    X_tonnetz = []
    
    for idx in trange(wavfiles.shape[0]):
        X_tonnetz.append(wave2tonnetz(wavfiles[idx, :], sr, max_pad_len=max_pad_len, 
                             normalize=normalize, padding_mode=padding_mode))
        pass
    
    X_tonnetz = np.array(X_tonnetz)
    
    # Saving Mels 
    np.save(output_path, X_tonnetz)
    
    pass

### RUNING

In [5]:
gen_tonnetz(INPUT_PATH, OUTPUT_PATH, SR, normalize=False)